# 装饰器

## 装饰器的简单示例

In [1]:
# 定义两个功能函数
def fss():
    print("发说说")

def ftp():
    print("发图片")
    
# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    fss()
else:
    ftp()

发图片


In [2]:
# 操作前提需要有登陆验证的操作
# 1. 直接在业务逻辑代码里面添加验证操作

# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    print("登陆验证...")
    fss()
else:
    print("登陆验证...")
    ftp()

# 由于业务逻辑代码非常多，所以每次调用功能函数之前，都需要做一次验证，所以代码冗余度比较大，且不好维护

登陆验证...
发图片


In [4]:
# 2. 直接在功能函数里面修改
# 定义两个功能函数
def checkLogin():
    print("登陆验证...")

def fss():
    checkLogin()
    print("发说说")

def ftp():
    checkLogin()
    print("发图片")
    
# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    fss()
else:
    ftp()
    
# 这样实现不适用于面向对象的思想，也没有遵循函数功能单一原则。

登陆验证...
发图片


In [ ]:
# 3. 将函数作为参数传递进来

# 定义两个功能函数
def checkLogin(func):
    print("登陆验证...")
    func()

def fss():
    print("发说说")

def ftp():
    print("发图片")
    
# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    # fss()
    checkLogin(fss)
else:
    # ftp()
    checkLogin(fss)
    
# 这样写的问题在于，业务逻辑代码还是发生了改变，需要更改所有业务逻辑代码

In [5]:
# 4. 采用闭包来不更改原功能函数，其实这已经实现了装饰器
# 如果不采用闭包，对象赋值时，会直接调用函数，如下：
"""
def checkLogin(func):
    print("登陆验证...")
    func()

def fss():
    print("发说说")
fss = checkLogin(fss)
"""


# 定义两个功能函数
def checkLogin(func):
    def inner():
        print("登陆验证...")
        func()
    return inner

def fss():
    print("发说说")
fss = checkLogin(fss)
    
def ftp():
    print("发图片")
ftp = checkLogin(ftp)
    
# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    fss()
else:
    ftp()

登陆验证...
发图片


In [6]:
# 5. python 语法糖实现装饰器

# 定义两个功能函数
def checkLogin(func):
    def inner():
        print("登陆验证...")
        func()
    return inner

@checkLogin
def fss():
    print("发说说")

@checkLogin
def ftp():
    print("发图片")
    
# 相关逻辑代码
btnIndex = 2
if btnIndex == 1:
    fss()
else:
    ftp()

登陆验证...
发图片


## 装饰器执行时间

In [7]:
def checkLogin(func):
    print("是否执行")
    def inner():
        print("登陆验证...")
        func()
    return inner

@checkLogin
def fss():
    print("发说说")

是否执行


## 函数装饰器

### 带参数的装饰器

In [2]:
def my_decorator(func):
    def wrapper(message):
        print('wrapper of decorator')
        func(message)
    return wrapper

@my_decorator
def greet(message):
	print(message)
    
greet('hello world')

wrapper of decorator
hello world


### 带自定义参数的装饰器

In [10]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decorator')
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(4)
def greet(message):
    print(message)
    
greet("hello world")

wrapper of decorator
hello world
wrapper of decorator
hello world
wrapper of decorator
hello world
wrapper of decorator
hello world


### 原函数还是原函数吗？

In [11]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decorator')
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(4)
def greet(message):
    print(message)

print(greet.__name__)

'wrapper'

In [37]:
def func2(func):
    print('x')
    def wrapper1():
        print('hello', end=" ")
        func()
    return wrapper1

# @func2
def func1():
    print('world')
    
    
print(func1.__name__)

x = func2(func1)

print(x.__name__)

y = x

print(y.__name__)

x()
y()

func1
x
wrapper1
wrapper1
hello world
hello world


In [34]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('wrapper of decorator')
        func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(message):
    print(message)
   
print(greet.__name__)	# greet
print(my_decorator.__name__)

greet
my_decorator


## 类装饰器

### 类中使用装饰器

In [1]:
import functools

class SimpleClass(object):
    @staticmethod
    def my_decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('wrapper of decorator')
            func(*args, **kwargs)
        return wrapper

@SimpleClass.my_decorator
def greet(message):
    print(message)
    
greet('hello world')

wrapper of decorator
hello world


### 装饰器装饰同一个类里的函数

In [5]:
import functools

class SimpleClass(object):
    def __init__(self):
        self.message_call = 0
    
    # @staticmethod  
    # 此处不能含有 staticmethod，因为装饰器需要传入 类实例
    # 也不能写成 my_decorator(self, func), 因为传入参数只有func
    def my_decorator(func):
        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):  # self,接收 SimpleClass 里的self,也就是类实例
            self.message_call += 1
            print('message_call is {}'.format(self.message_call))
            return func(self, *args, **kwargs)
        return wrapper

    @my_decorator
    def greet(self, message):
        print(message)
    
test = SimpleClass()
test.greet('hello world')
test.greet('hello world')

message_call is 1
hello world
message_call is 2
hello world


In [6]:
import functools

class SimpleClass(object):
    def __init__(self):
        self.message_call = 0
    
    # @staticmethod  
    # 此处不能含有 staticmethod，因为装饰器需要传入 类实例
    # 也不能写成 my_decorator(self, func), 因为传入参数只有func
    def my_decorator(func):
        print("弄懂调用次数")
        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):  # self,接收 SimpleClass 里的self,也就是类实例
            self.message_call += 1
            print('message_call is {}'.format(self.message_call))
            return func(self, *args, **kwargs)
        return wrapper

    @my_decorator
    def greet1(self, message):
        print(message)
        
    @my_decorator
    def greet2(self, message):
        print(message)
    
test = SimpleClass()

弄懂调用次数
弄懂调用次数


### 定义一个类装饰器，装饰类中的函数，默认调用__get__方法

实际上把类方法变成属性了，还记得类属性装饰器吧，@property

下面自已做一个property

In [2]:
class Decrator(object):
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, owner):
        '''
        instance:代表实例，sum中的self
        owner：代表类本身，Test类
        
        '''
        print('调用的是get函数')
        return self.func(instance)     # instance就是Test类的self


class Test(object):
    def __init__(self):
        self.result = 0

    @Decrator
    def sum(self):
        print('There is the Func in the Class !')

t = Test()
print(t.sum)            # 众所周知，属性是不加括号的,sum真的变成了属性

调用的是get函数
There is the Func in the Class !
None


 做一个求和属性sum，统计所有输入的数字的和

In [8]:
class Decrator(object):
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, owner):
        print('调用的是get函数')
        return self.func(instance)


class Test(object):
    def __init__(self, *args, **kwargs):
        self.value_list = []
        if args:
            for i in args:
                if str(i).isdigit():
                    self.value_list.append(i)
        if kwargs:
            for v in kwargs.values():
                if str(v).isdigit():
                    self.value_list.append(v)

    @Decrator
    def sum(self):
        result = 0
        print(self.value_list)
        for i in self.value_list:
            result += i

        return result


t = Test(1, 2, 3, 4, 5, 6, 7, 8, i=9, ss=10, strings='lll')

print(t.sum)

调用的是get函数
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
55


## 装饰器的嵌套

### 从上到下装饰，从下到上执行

In [9]:
import functools
def my_decorator1(func):
    print("xx1")
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator1')
        func(*args, **kwargs)
    return wrapper

def my_decorator2(func):
    print("xx2")
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator2')
        func(*args, **kwargs)
        return wrapper
    
@my_decorator1
@my_decorator2
def greet(message):
	print(message)

xx2
xx1
